In [1]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from matplotlib.animation import FuncAnimation
import random
from math import ceil

import matplotlib as mpl 
mpl.rcParams['animation.ffmpeg_path'] = r'...\\ffmpeg-2023-04-24-git-2aad9765ef-essentials_build\\bin\\ffmpeg.exe'

In [2]:
user = pd.read_csv('user1_preprocessed_4.csv', index_col=0)
user = user.filter(['X', 'Y', 'PAM_Val', 'delta_time', 'seq_num'])
user.index = pd.to_datetime(user.index)

print(user)
print(user.index.dtype)

                              X    Y  PAM_Val  delta_time  seq_num
Time                                                              
2021-09-10 11:59:42.515770  518  381        4         0.0        0
2021-09-10 11:59:42.523750  511  388        4      7980.0        0
2021-09-10 11:59:42.531727  509  393        4      7977.0        0
2021-09-10 11:59:42.539705  505  397        4      7978.0        0
2021-09-10 11:59:42.547684  501  399        4      7979.0        0
...                         ...  ...      ...         ...      ...
2021-09-14 18:31:29.481415  424  676        1      4985.0       32
2021-09-14 18:31:29.498370  429  673        1     16955.0       32
2021-09-14 18:31:29.514327  432  671        1     15957.0       32
2021-09-14 18:31:29.666919  432  671        1    152592.0       32
2021-09-14 18:31:29.747703  432  671        1     80784.0       32

[844221 rows x 5 columns]
datetime64[ns]


In [3]:
#plt.plot(user['Time'][:1000], user['delta_time'][:1000].values)

In [4]:
user_grouped = user.groupby(by='seq_num')

sequences = []
for seq_num, seq in user_grouped:
    seq = seq.loc[seq.index > seq.index[-1] - pd.Timedelta(pd.offsets.Second(60))]
    sequences.append(seq.drop(columns=['seq_num']))

print(sequences[0])

                               X    Y  PAM_Val  delta_time
Time                                                      
2021-09-10 12:03:02.960078  1103  603        4    883213.0
2021-09-10 12:03:02.968056  1103  602        4      7978.0
2021-09-10 12:03:02.976035  1103  601        4      7979.0
2021-09-10 12:03:32.540002  1103  600        4    563967.0
2021-09-10 12:03:32.547980  1103  600        4      7978.0
...                          ...  ...      ...         ...
2021-09-10 12:03:49.435837   768  301        4      7981.0
2021-09-10 12:03:49.442819   768  300        4      6982.0
2021-09-10 12:03:49.467751   768  300        4     24932.0
2021-09-10 12:03:49.483706   768  299        4     15955.0
2021-09-10 12:03:49.596407   768  299        4    112701.0

[646 rows x 4 columns]


In [5]:
def create_animation(sequence, file_name, interval, speed_multiplier):
    fig = plt.figure()
    ax = plt.axes(xlim=(0, 2560), ylim=(0, 1440))
    line, = ax.plot([], [], lw=1)

    plotcols = ['red', 'blue']
    lines = []
    for index in range(2):
        lobj = ax.plot([], [], lw=1, color=plotcols[index])[0]
        lines.append(lobj)
        
    def init():
        for line in lines:
            line.set_data([], [])
        return lines
    def animate(i):
        data = sequence.loc[sequence.index <= sequence.index[0] + pd.Timedelta(pd.offsets.Milli(int(interval * i * speed_multiplier)))]
        #curr_data = sequence[i-10:i]

        lines[0].set_data(data['X'].tolist(), data['Y'].tolist())
        #lines[1].set_data(curr_data['X'].tolist(), curr_data['Y'].tolist())


        return line,

    duration = int(ceil((sequence.index[-1] - sequence.index[0]).total_seconds() * 1e3))
    anim = FuncAnimation(fig, animate, init_func=init, frames=duration // (interval * speed_multiplier), interval=interval, blit=True)

    anim.save(file_name+'_'+str(speed_multiplier)+'x'+'.mp4')

#print(sequences[0])
speed_multiplier = 1
#create_animation(sequence=sequences[1], file_name='output_color_test', interval=50, speed_multiplier=speed_multiplier)
for index, sequence in enumerate(sequences):
    create_animation(sequence=sequence, file_name='output'+str(index), interval=50, speed_multiplier=speed_multiplier)
    print(str(index), 'done')

0 done
1 done
2 done
3 done
4 done
5 done
6 done
7 done
8 done
9 done
10 done
11 done
12 done
